# text-generator-combinator
- aka *gcom*
- a text generator combinator inspired by arcsecond
- used to generate text samples for NLU models like those used by LUIS

In [1]:
G = {}

{}

In [2]:
generatorDictionary = {};
addGenerator = function (dictionary, name, generator) {
    dictionary[name] = generator;
}

[Function: addGenerator]

In [44]:
generatorDictionary

{}

In [481]:
G.str = (str) => {
    return {
        run: (context) => {
            if (typeof context !== 'undefined') {
                console.log(`str context:`, context);
            }
            return str;
        }
    }
}

[Function]

In [482]:
testGen = G.str('hello');

{ run: [Function: run] }

In [483]:
testGen.run()

'hello'

In [484]:
addGenerator(generatorDictionary, 'test', testGen);

In [485]:
generatorDictionary['test'].run()

'hello'

In [486]:
G.sequenceOf = function(generators, options) {
    return {
        run: () => {
            let results = [];
            let delimiter = ' ';
            let context = { name: 'SEQUENCE-OF-CONTEXT'};
            if (options) {
                if (options.delimiter !== undefined) {
                     delimiter = options.delimiter
                }
            }
            generators.forEach(generator => {
                let text = generator.run(context);
                if (text) {
                    results.push(text);
                }
            });
            return results.join(delimiter);
        }
    }
}

[Function]

In [487]:
testSequenceGen = G.sequenceOf([testGen, testGen], { delimiter: ' '})

{ run: [Function: run] }

In [488]:
testSequenceGen

{ run: [Function: run] }

In [489]:
testSequenceGen.run();

str context: { name: 'SEQUENCE-OF-CONTEXT' }
str context: { name: 'SEQUENCE-OF-CONTEXT' }


'hello hello'

In [490]:
G.shuffleArray = (array) => {
    for (let i = array.length - 1; i > 0; i--) {
        const j = Math.floor(Math.random() * (i + 1));
        [array[i], array[j]] = [array[j], array[i]];
    }
}

[Function]

In [491]:
G.strings = (strings, options) => {
    let cl = (strings) => {
        let i = 0;
        let length = strings.length;
        let mode = options ? options.mode : '';
        if (mode === 'shuffle') {
            G.shuffleArray(strings);
        }
        return {
            run: () => {
                let result = strings[i++];
                if (i >= length) {
                    i = 0;
                }
                if (mode === 'random') {
                    let randomIndex = Math.floor(Math.random() * length);
                    result =  strings[randomIndex]
                }
                return result;
            }
        }
    }
    return cl(strings);
}

[Function]

In [492]:
worldGen = G.strings(['world', 'galaxy', 'universe'], { mode: 'shuffle' });

{ run: [Function: run] }

In [493]:
for (let i=0; i<10; i++) {
    console.log(worldGen.run());
}

galaxy
universe
world
galaxy
universe
world
galaxy
universe
world
galaxy


In [494]:
helloSequenceGen = G.sequenceOf([testGen, worldGen])

{ run: [Function: run] }

In [495]:
for (let i=0; i<10; i++) {
    console.log(helloSequenceGen.run());
}

str context: { name: 'SEQUENCE-OF-CONTEXT' }
hello universe
str context: { name: 'SEQUENCE-OF-CONTEXT' }
hello world
str context: { name: 'SEQUENCE-OF-CONTEXT' }
hello galaxy
str context: { name: 'SEQUENCE-OF-CONTEXT' }
hello universe
str context: { name: 'SEQUENCE-OF-CONTEXT' }
hello world
str context: { name: 'SEQUENCE-OF-CONTEXT' }
hello galaxy
str context: { name: 'SEQUENCE-OF-CONTEXT' }
hello universe
str context: { name: 'SEQUENCE-OF-CONTEXT' }
hello world
str context: { name: 'SEQUENCE-OF-CONTEXT' }
hello galaxy
str context: { name: 'SEQUENCE-OF-CONTEXT' }
hello universe


In [402]:
helloGen = G.strings(['hello', 'greetings', 'yo', 'wassup']);

{ run: [Function: run] }

In [403]:
helloSequenceGen = G.sequenceOf([helloGen, worldGen])

{ run: [Function: run] }

In [404]:
for (let i=0; i<10; i++) {
    console.log(helloSequenceGen.run());
}

hello galaxy
greetings world
yo universe
wassup galaxy
hello world
greetings universe
yo galaxy
wassup world
hello universe
greetings galaxy


In [405]:
G.optional = (generator, options) => {
    let cl = (generator, options) => {
        options = options || {};
        let active = false;
        return {
            run: () => {
                let result = '';
                if (options.random) {
                    let rnd = Math.random();
                    if (rnd <= options.random) {
                        result = generator.run();
                    }
                } else if (active) {
                    result = generator.run();
                }
                active = !active;
                return result;
            }
        }
    }
    return cl(generator, options);
}

[Function]

In [406]:
helloGen.run()

'yo'

In [407]:
optionalHello = G.optional(helloGen, { random: .3 });

{ run: [Function: run] }

In [415]:
optionalHello.run();

'hello'

In [412]:
peopleGen = G.strings([', friend', ', everyone', ', gang', ', my peeps']);

{ run: [Function: run] }

In [413]:
peopleGen.run()

', friend'

In [214]:
optionalHelloPeople = G.optional(peopleGen, { random: .3 });

{ run: [Function: run] }

In [215]:
optionalHelloPeople.run()

', everyone'

In [216]:
helloPeopleSequenceGen = G.sequenceOf([helloGen, worldGen, optionalHelloPeople])

[ { run: [Function: run] },
  { run: [Function: run] },
  { run: [Function: run] } ]


{ run: [Function: run] }

In [424]:
helloPeopleSequenceGen.run()

'wassup universe , friend'

### gramar example
```
V_RECORDING = (
	[record?(s|(ing))]|([watch?(ing)] and [listen?(ing)])|([listen?(ing)] and [watch?(ing)])|
	([see?(ing)] and [hear?(ing)])|([hear?(ing)] and [see?(ing)])|
	([hav(e|(ing))]|[stor(e|(ing))] ?a record ?of everything|what ?((we say)|(you hear)))
);
```

#### example
```
[record?(s|(ing))]
```

In [224]:
endings = G.strings(['s', 'ing']);

{ run: [Function: run] }

In [426]:
endings.run()

's'

In [226]:
optionalEndings = G.optional(endings);

{ run: [Function: run] }

In [241]:
recordString = G.str('record')

{ run: [Function: run] }

In [427]:
recordingGen = G.sequenceOf([recordString, optionalEndings], { delimiter: ''})

{ run: [Function: run] }

In [428]:
for (let i=0; i<10; i++) {
    console.log(recordingGen.run());
}

recording
record
records
record
recording
record
records
record
recording
record


#### example
```
?a record ?of everything|what ?((we say)|(you hear)))
```

In [429]:
phraseGen = G.sequenceOf([
    G.optional(G.str('a')),
    G.str('record'),
    G.sequenceOf([
    G.optional(G.str('of')),
        G.strings(['everything', 'what'], { mode: 'random' }),
    ]),
    G.optional(G.strings(['we say', 'you hear'], { mode: 'random' }), { random: .75 })
])

{ run: [Function: run] }

In [430]:
for (let i=0; i<20; i++) {
    console.log(phraseGen.run());
}

record everything you hear
a record of what you hear
record everything we say
a record of what
record everything
a record of everything we say
record everything you hear
a record of everything you hear
record everything
a record of what you hear
record everything you hear
a record of everything
record what you hear
a record of everything you hear
record what we say
a record of everything you hear
record everything you hear
a record of what you hear
record what we say
a record of what you hear


In [352]:
G.unique = function(generator, count, options) {
    count = count || 10;
    //return {
        //run: () => {
            let resultsDictionary = {};
            let maxAttempts = 10;
            if (options) {
                if (options.attempts !== undefined) {
                     maxAttempts = options.attempts
                }
            }
            let attemptCount = 0;
            let done = false;
            while (attemptCount < maxAttempts && !done) {
                let text = generator.run();
                if (!resultsDictionary[text]) {
                    resultsDictionary[text] = text;
                    attemptCount = 0;
                } else {
                    attemptCount++;
                }
                
                let keys = Object.keys(resultsDictionary);
                if (keys.length == count) {
                    done = true;
                }
            }

            let keys = Object.keys(resultsDictionary).sort()
            return {
                found: keys.length,
                phrases: keys,
                attempts: attemptCount,
            }
        //}
    //}
}

[Function]

In [353]:
G.unique(G.str('hello'))

{ found: 1, phrases: [ 'hello' ], attempts: 10 }

In [354]:
G.unique(G.strings(['hello', 'greetings']))

{ found: 2, phrases: [ 'greetings', 'hello' ], attempts: 10 }

In [431]:
G.unique(phraseGen, 20, { attempts: 100 })

{ found: 12,
  phrases:
   [ 'a record of everything',
     'a record of everything we say',
     'a record of everything you hear',
     'a record of what',
     'a record of what we say',
     'a record of what you hear',
     'record everything',
     'record everything we say',
     'record everything you hear',
     'record what',
     'record what we say',
     'record what you hear' ],
  attempts: 100 }

### rule example
```
TopRule = $* (
    $food {book='food'} {item = food._item}
) $*;


reserve = reserve | order | book | make;

food = $reserve $* $sandwich {_item = sandwich._type};
sandwich = ([sandwich?(es)] | ([sub?s] [thing?(s|y|(ies))]) | panini) {_type = 'sandwich'};
```

```
([sub?s] [thing?(s|y|(ies))])
```

In [376]:
subGen = G.sequenceOf([
    G.strings(['reserve', 'order', 'book', 'make']),
    G.str('me a'),
    G.sequenceOf([
        G.str('sub'),
        G.optional(G.str('s'), { random: 0.3 })
    ], { delimiter: ''}),
     G.sequenceOf([
        G.str('thing'),
        G.optional(G.strings(['y','s','ies']))
    ], { delimiter: ''}),
])

{ run: [Function: run] }

In [377]:
for (let i=0; i<10; i++) {
    console.log(subGen.run());
}

reserve me a subs thing
order me a subs thingy
book me a sub thing
make me a subs things
reserve me a sub thing
order me a sub thingies
book me a sub thing
make me a subs thingy
reserve me a sub thing
order me a subs things


In [381]:
G.unique(subGen, 20, 100)

{ found: 15,
  phrases:
   [ 'book me a sub thing',
     'book me a subs thing',
     'make me a sub thingies',
     'make me a sub things',
     'make me a sub thingy',
     'make me a subs things',
     'make me a subs thingy',
     'order me a sub thingies',
     'order me a sub things',
     'order me a sub thingy',
     'order me a subs thingies',
     'order me a subs things',
     'order me a subs thingy',
     'reserve me a sub thing',
     'reserve me a subs thing' ],
  attempts: 10 }